Queries data from WRDS

In [138]:
# Standard Imports
import numpy as np
import pandas as pd
import pickle
import datetime
import itertools
import wrds
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import linear_model as lm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

# Import Custom Utilities
import thesis_utils as utils

# Connect to WRDS
db = wrds.Connection(wrds_username='akwlau')

Loading library list...
Done


PERMNO ID codes

In [139]:
# Get permnos
permnos = db.get_table(library='crsp', table='msp500list')
permnos['permno'] = permnos['permno'].astype(int) # cast to int


Fama French Industries Categorizations

In [140]:
industries = [None] * 12
industries[1] = list(range(100,1000))+list(range(2000,2400))+list(range(2700,2750))+list(range(2770,2800))+list(range(3100,3200))+list(range(3940,3990))
industries[2] = list(range(2500,2520))+list(range(2590,2600))+list(range(3630,3660))+list(range(3710,3712))+list(range(3714,3715))+list(range(3716,3717))+list(range(3750,3752))+list(range(3792,3793))+list(range(3900,3940))+list(range(3990,4000))
industries[3] = list(range(2520,2590))+list(range(2600,2700))+list(range(2750,2770))+list(range(3000,3100))+list(range(3200,3570))+list(range(3580,3630))+list(range(3700,3710))+list(range(3712,3714))+list(range(3715,3716))+list(range(3717,3750))+list(range(3752,3792))+list(range(3793,3800))+list(range(3830,3840))+list(range(3860,3900))
industries[4] = list(range(1200,1400))+list(range(2900,3000))
industries[5] = list(range(2800,2830))+list(range(2840,2900))
industries[6] = list(range(3570,3580))+list(range(3660,3693))+list(range(3694,3700))+list(range(3810,3830))+list(range(7370,7380))
industries[7] = list(range(4800,4900))
industries[8] = list(range(4900,4950))
industries[9] = list(range(5000,6000))+list(range(7200,7300))+list(range(7600,7700))
industries[10] = list(range(2830,2840))+list(range(3693,3694))+list(range(3840,3860))+list(range(8000,8100))
industries[11] = list(range(6000,7000))

ind_names = [None, 'NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth', 'Fin']
industries_dict = {}
for idx, sic_list in enumerate(industries):
    if sic_list is not None:
        for s in sic_list:
            industries_dict.update({s:ind_names[idx]})

Query and pickle data

In [141]:
for i in range(30): # 30
    year = 1993 + i
    # year = 2020

    # Get stock data
    first_trade_date_string = f'{year}-01-01'
    first_trade_date = pd.Timestamp(first_trade_date_string).date() # date of start of trade period

    first_trade_date_minus_three = first_trade_date - pd.offsets.DateOffset(years=3, days=10) # 750 + buffer
    first_trade_date_plus_one = first_trade_date + pd.offsets.DateOffset(years=1, days=10) # 250 + buffer
    data_start_date = first_trade_date_minus_three.strftime('%m/%d/%Y')
    data_end_date = first_trade_date_plus_one.strftime('%m/%d/%Y')

    # Stock time series + list of current permnos
    data, current_permnos = utils.get_stock_data(db, permnos, data_start_date, data_end_date, first_trade_date)
    # data['sic'] = [str(s)[:2] for s in data['hsiccd']] # add sic codes
    data['ind'] = data.apply(lambda row: industries_dict.get(row['hsiccd'], 'Other'), axis=1) # create industries

    pickle.dump((data, current_permnos), open( f"Data/data_for{first_trade_date_string}_v2.p", "wb" ))
    print(f'{year} Data Done')
    # print(data['hsiccd'].isna().sum(), (data['hsiccd'] == 0).sum())

1993 Data Done
1994 Data Done
1995 Data Done
1996 Data Done
1997 Data Done
1998 Data Done
1999 Data Done
2000 Data Done
2001 Data Done
2002 Data Done
2003 Data Done
2004 Data Done
2005 Data Done
2006 Data Done
2007 Data Done
2008 Data Done
2009 Data Done
2010 Data Done
2011 Data Done
2012 Data Done
2013 Data Done
2014 Data Done
2015 Data Done
2016 Data Done
2017 Data Done
2018 Data Done
2019 Data Done
2020 Data Done
2021 Data Done
2022 Data Done
